# Python 后端开发工具链全攻略

## 学习目标
- 掌握现代 Python 开发环境的搭建和配置
- 学会使用主流的包管理和虚拟环境工具
- 熟悉代码质量保证工具（格式化、静态分析、测试）
- 了解调试技巧和性能分析工具
- 掌握版本控制和 CI/CD 基础
- 建立高效的开发工作流

## 为什么工具链很重要？

在现代软件开发中，**工具链就是生产力**。一个良好的工具链可以：

- **提高开发效率**：自动化重复性任务
- **保证代码质量**：静态分析、格式化、测试
- **团队协作**：统一的开发环境和规范
- **快速部署**：CI/CD 自动化流程
- **问题定位**：调试和监控工具

> 💡 **PyBackendPro 经验**：投入时间学习和配置工具链，会在长期开发中获得数倍的回报。专业开发者与业余爱好者的区别往往体现在工具使用的熟练程度上。


In [ ]:
import sys
import os
import subprocess
import platform
from pathlib import Path
import json
from typing import List, Dict, Any
import shutil
import venv

print("🛠️  Python 后端开发工具链指南")
print("=" * 50)
print(f"Python 版本: {sys.version}")
print(f"操作系统: {platform.system()} {platform.release()}")
print(f"架构: {platform.machine()}")
print(f"当前工作目录: {os.getcwd()}")
print("=" * 50)

def run_command(command: str, capture_output: bool = True) -> Dict[str, Any]:
    """
    安全地执行命令行命令
    
    Args:
        command: 要执行的命令
        capture_output: 是否捕获输出
        
    Returns:
        执行结果字典
    """
    try:
        result = subprocess.run(
            command.split(),
            capture_output=capture_output,
            text=True,
            timeout=30
        )
        return {
            'success': result.returncode == 0,
            'stdout': result.stdout.strip() if capture_output else '',
            'stderr': result.stderr.strip() if capture_output else '',
            'returncode': result.returncode
        }
    except subprocess.TimeoutExpired:
        return {'success': False, 'error': '命令执行超时'}
    except FileNotFoundError:
        return {'success': False, 'error': '命令未找到'}
    except Exception as e:
        return {'success': False, 'error': str(e)}

def check_tool_availability():
    """检查开发工具的可用性"""
    tools = {
        'pip': 'pip --version',
        'git': 'git --version',
        'python': 'python --version',
    }
    
    print("检查开发工具可用性:")
    for tool, command in tools.items():
        result = run_command(command)
        status = "✅" if result['success'] else "❌"
        version = result['stdout'].split('\n')[0] if result['success'] else "未安装"
        print(f"  {status} {tool}: {version}")

check_tool_availability()


## 1. 包管理和虚拟环境

### Python 包管理器对比

| 工具 | 特点 | 适用场景 | 学习难度 |
|------|------|----------|----------|
| **pip** | Python 内置，简单直接 | 基础项目、学习 | ⭐ |
| **pipenv** | 自动虚拟环境管理 | 中小型项目 | ⭐⭐ |
| **poetry** | 现代依赖解析，构建打包 | 大型项目、库开发 | ⭐⭐⭐ |
| **conda** | 跨语言，科学计算 | 数据科学、机器学习 | ⭐⭐ |

### 虚拟环境的重要性
- **依赖隔离**：不同项目使用不同版本的库
- **环境一致性**：开发、测试、生产环境保持一致
- **避免冲突**：防止全局包污染
- **可重现性**：通过 requirements.txt 重建环境

### 推荐工作流
1. **学习阶段**：使用 pip + venv
2. **项目开发**：使用 pipenv 或 poetry
3. **生产部署**：使用 Docker 容器化


In [ ]:
class VirtualEnvManager:
    """
    虚拟环境管理器 - 演示虚拟环境操作
    """
    
    def __init__(self, project_name: str = "demo_project"):
        self.project_name = project_name
        self.venv_path = Path.cwd() / f".venv_{project_name}"
        
    def create_venv(self) -> bool:
        """创建虚拟环境"""
        try:
            if self.venv_path.exists():
                print(f"⚠️  虚拟环境已存在: {self.venv_path}")
                return True
            
            print(f"🔨 创建虚拟环境: {self.venv_path}")
            venv.create(self.venv_path, with_pip=True)
            
            print("✅ 虚拟环境创建成功")
            return True
        except Exception as e:
            print(f"❌ 创建虚拟环境失败: {e}")
            return False
    
    def get_activation_command(self) -> str:
        """获取激活命令"""
        if platform.system() == "Windows":
            return str(self.venv_path / "Scripts" / "activate.bat")
        else:
            return f"source {self.venv_path}/bin/activate"
    
    def get_pip_path(self) -> Path:
        """获取虚拟环境中的 pip 路径"""
        if platform.system() == "Windows":
            return self.venv_path / "Scripts" / "pip.exe"
        else:
            return self.venv_path / "bin" / "pip"
    
    def install_package(self, package: str) -> bool:
        """在虚拟环境中安装包"""
        pip_path = self.get_pip_path()
        if not pip_path.exists():
            print(f"❌ pip 不存在: {pip_path}")
            return False
        
        print(f"📦 安装包: {package}")
        result = run_command(f"{pip_path} install {package}")
        
        if result['success']:
            print(f"✅ 成功安装: {package}")
            return True
        else:
            print(f"❌ 安装失败: {result.get('stderr', 'Unknown error')}")
            return False
    
    def list_packages(self) -> List[str]:
        """列出已安装的包"""
        pip_path = self.get_pip_path()
        if not pip_path.exists():
            return []
        
        result = run_command(f"{pip_path} list")
        if result['success']:
            lines = result['stdout'].split('\n')[2:]  # 跳过头部信息
            packages = []
            for line in lines:
                if line.strip():
                    parts = line.split()
                    if len(parts) >= 2:
                        packages.append(f"{parts[0]}=={parts[1]}")
            return packages
        return []
    
    def generate_requirements(self) -> str:
        """生成 requirements.txt 内容"""
        packages = self.list_packages()
        return '\n'.join(packages)
    
    def cleanup(self) -> bool:
        """清理虚拟环境"""
        try:
            if self.venv_path.exists():
                shutil.rmtree(self.venv_path)
                print(f"🗑️  虚拟环境已删除: {self.venv_path}")
            return True
        except Exception as e:
            print(f"❌ 删除失败: {e}")
            return False

# 虚拟环境演示
print("\n=== 虚拟环境管理演示 ===")
venv_manager = VirtualEnvManager("backend_demo")

# 创建虚拟环境
if venv_manager.create_venv():
    print(f"\n激活命令: {venv_manager.get_activation_command()}")
    
    # 安装一些常用包
    packages_to_install = ["requests", "fastapi"]
    for package in packages_to_install:
        venv_manager.install_package(package)
    
    # 列出已安装的包
    print(f"\n📋 已安装的包:")
    installed_packages = venv_manager.list_packages()
    for package in installed_packages[:10]:  # 显示前10个
        print(f"  - {package}")
    
    # 生成 requirements.txt
    requirements_content = venv_manager.generate_requirements()
    print(f"\n📄 requirements.txt 内容预览:")
    print(requirements_content[:300] + "..." if len(requirements_content) > 300 else requirements_content)


## 2. 代码质量工具

### 代码格式化工具

| 工具 | 特点 | 配置复杂度 | 团队推荐 |
|------|------|------------|----------|
| **Black** | 无需配置，固执己见 | 极低 | ⭐⭐⭐⭐⭐ |
| **autopep8** | 基于 PEP 8，可配置 | 中等 | ⭐⭐⭐ |
| **yapf** | Google 出品，高度可配置 | 高 | ⭐⭐ |

### 静态分析工具

| 工具 | 主要功能 | 适用场景 |
|------|----------|----------|
| **flake8** | 语法检查、风格检查 | 基础项目 |
| **pylint** | 全面的代码分析 | 大型项目 |
| **mypy** | 类型检查 | 类型注解项目 |
| **bandit** | 安全漏洞检查 | 安全要求高的项目 |

### 一体化解决方案
- **pre-commit**：Git hooks 管理
- **tox**：多环境测试
- **ruff**：新一代快速 linter（Rust 编写）


In [ ]:
class CodeQualityManager:
    """
    代码质量管理器 - 演示代码质量工具的使用
    """
    
    def __init__(self, project_path: Path = None):
        self.project_path = project_path or Path.cwd()
        self.tools_config = {
            'black': {
                'command': 'black --check --diff',
                'description': '代码格式化检查'
            },
            'flake8': {
                'command': 'flake8',
                'description': '代码风格和语法检查'
            },
            'mypy': {
                'command': 'mypy',
                'description': '类型检查'
            },
            'bandit': {
                'command': 'bandit -r',
                'description': '安全漏洞检查'
            }
        }
    
    def create_sample_code_files(self):
        """创建示例代码文件用于演示"""
        
        # 创建一个有问题的 Python 文件
        bad_code = '''
import os,sys
import requests

def bad_function( x,y ):
    # 这是一个有问题的函数
    result=x+y
    print("Result is: "+str(result))
    return result

class   BadClass:
    def __init__(self,name):
        self.name=name
    
    def get_data(self):
        # 安全问题：硬编码密码
        password = "admin123"
        url = "http://api.example.com/data"
        response = requests.get(url, auth=("admin", password))
        return response.json()

def unused_function():
    pass

if __name__=="__main__":
    obj=BadClass("test")
    result=bad_function(1,2)
'''
        
        # 创建一个良好的 Python 文件
        good_code = '''"""
示例模块：演示良好的 Python 代码风格
"""
import os
import sys
from typing import Dict, Any, Optional

import requests


class DataFetcher:
    """数据获取器类"""
    
    def __init__(self, name: str) -> None:
        """
        初始化数据获取器
        
        Args:
            name: 获取器名称
        """
        self.name = name
        self._api_url = os.environ.get("API_URL", "https://api.example.com")
    
    def get_data(self, endpoint: str) -> Optional[Dict[str, Any]]:
        """
        获取数据
        
        Args:
            endpoint: API 端点
            
        Returns:
            API 响应数据
        """
        try:
            # 使用环境变量存储敏感信息
            api_key = os.environ.get("API_KEY")
            if not api_key:
                raise ValueError("API_KEY 环境变量未设置")
            
            headers = {"Authorization": f"Bearer {api_key}"}
            response = requests.get(
                f"{self._api_url}/{endpoint}",
                headers=headers,
                timeout=30
            )
            response.raise_for_status()
            return response.json()
            
        except requests.RequestException as e:
            print(f"请求失败: {e}")
            return None


def calculate_sum(x: int, y: int) -> int:
    """
    计算两个数的和
    
    Args:
        x: 第一个数
        y: 第二个数
        
    Returns:
        两数之和
    """
    result = x + y
    print(f"计算结果: {result}")
    return result


if __name__ == "__main__":
    fetcher = DataFetcher("示例获取器")
    result = calculate_sum(1, 2)
'''
        
        # 写入文件
        (self.project_path / "bad_example.py").write_text(bad_code, encoding='utf-8')
        (self.project_path / "good_example.py").write_text(good_code, encoding='utf-8')
        
        print("✅ 示例代码文件已创建")
        print(f"  - {self.project_path / 'bad_example.py'}")
        print(f"  - {self.project_path / 'good_example.py'}")
    
    def create_config_files(self):
        """创建代码质量工具的配置文件"""
        
        # pyproject.toml 配置
        pyproject_config = '''[tool.black]
line-length = 88
target-version = ['py38']
include = '\\.pyi?$'

[tool.isort]
profile = "black"
multi_line_output = 3

[tool.mypy]
python_version = "3.8"
warn_return_any = true
warn_unused_configs = true
disallow_untyped_defs = true

[tool.bandit]
exclude_dirs = ["tests"]
skips = ["B101"]
'''
        
        # flake8 配置
        flake8_config = '''[flake8]
max-line-length = 88
extend-ignore = E203, W503
exclude = 
    .git,
    __pycache__,
    .venv,
    .eggs,
    *.egg
per-file-ignores =
    __init__.py:F401
'''
        
        # pre-commit 配置
        precommit_config = '''repos:
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.4.0
    hooks:
      - id: trailing-whitespace
      - id: end-of-file-fixer
      - id: check-yaml
      - id: check-added-large-files

  - repo: https://github.com/psf/black
    rev: 23.1.0
    hooks:
      - id: black

  - repo: https://github.com/pycqa/flake8
    rev: 6.0.0
    hooks:
      - id: flake8

  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.0.1
    hooks:
      - id: mypy
'''
        
        # 写入配置文件
        (self.project_path / "pyproject.toml").write_text(pyproject_config, encoding='utf-8')
        (self.project_path / ".flake8").write_text(flake8_config, encoding='utf-8')
        (self.project_path / ".pre-commit-config.yaml").write_text(precommit_config, encoding='utf-8')
        
        print("✅ 配置文件已创建")
        print("  - pyproject.toml")
        print("  - .flake8")
        print("  - .pre-commit-config.yaml")
    
    def demonstrate_tool_usage(self):
        """演示工具使用"""
        print("\n=== 代码质量工具演示 ===")
        
        # 检查工具是否已安装
        tools_status = {}
        for tool in ['black', 'flake8', 'mypy', 'bandit']:
            result = run_command(f"{tool} --version")
            tools_status[tool] = result['success']
            status = "✅" if result['success'] else "❌"
            print(f"{status} {tool}: {'已安装' if result['success'] else '未安装'}")
        
        print(f"\n📝 分析文件:")
        for file in ['bad_example.py', 'good_example.py']:
            if (self.project_path / file).exists():
                print(f"  - {file}")
        
        # 模拟工具输出（因为可能没有安装这些工具）
        print(f"\n🔍 代码质量检查结果预览:")
        print(f"")
        print(f"Black (代码格式化):")
        print(f"  bad_example.py: ❌ 格式不符合规范，需要重新格式化")
        print(f"  good_example.py: ✅ 格式正确")
        print(f"")
        print(f"Flake8 (代码风格检查):")
        print(f"  bad_example.py: ❌ 发现 8 个问题")
        print(f"    - E401: 多个导入应分行")
        print(f"    - E225: 操作符周围缺少空格")
        print(f"    - E302: 期望有 2 个空行")
        print(f"  good_example.py: ✅ 无问题")
        print(f"")
        print(f"Mypy (类型检查):")
        print(f"  bad_example.py: ❌ 缺少类型注解")
        print(f"  good_example.py: ✅ 类型注解完整")
        print(f"")
        print(f"Bandit (安全检查):")
        print(f"  bad_example.py: ⚠️  发现硬编码密码")
        print(f"  good_example.py: ✅ 无安全问题")

# 代码质量工具演示
code_manager = CodeQualityManager()

# 创建示例文件
code_manager.create_sample_code_files()

# 创建配置文件
code_manager.create_config_files()

# 演示工具使用
code_manager.demonstrate_tool_usage()


## 3. 测试框架和调试工具

### 主流 Python 测试框架

| 框架 | 特点 | 学习曲线 | 推荐场景 |
|------|------|----------|----------|
| **unittest** | Python 内置，类似 JUnit | 中等 | 传统项目、教学 |
| **pytest** | 简洁语法，插件丰富 | 低 | 现代项目推荐 ⭐⭐⭐⭐⭐ |
| **nose2** | unittest 扩展 | 低 | 历史项目迁移 |

### 测试类型和工具

- **单元测试**：pytest, unittest
- **集成测试**：pytest-django, pytest-flask
- **API 测试**：httpx, requests-mock
- **性能测试**：locust, pytest-benchmark
- **覆盖率测试**：coverage.py, pytest-cov

### 调试工具对比

| 工具 | 类型 | 特点 | 使用场景 |
|------|------|------|----------|
| **pdb** | 命令行调试器 | Python 内置 | 简单调试 |
| **ipdb** | 增强版 pdb | IPython 集成 | 交互式调试 |
| **VS Code Debugger** | 图形化调试器 | 可视化界面 | 复杂调试 |
| **PyCharm Debugger** | IDE 调试器 | 功能完整 | 专业开发 |

### 性能分析工具

- **cProfile**：Python 内置性能分析器
- **line_profiler**：逐行性能分析
- **memory_profiler**：内存使用分析
- **py-spy**：生产环境性能采样


In [ ]:
class TestingFrameworkDemo:
    """
    测试框架演示类
    """
    
    def __init__(self):
        self.project_path = Path.cwd()
    
    def create_test_examples(self):
        """创建测试示例文件"""
        
        # 被测试的代码
        main_code = '''"""
计算器模块 - 用于演示测试
"""
from typing import Union

class Calculator:
    """简单计算器类"""
    
    def add(self, a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
        """加法运算"""
        return a + b
    
    def subtract(self, a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
        """减法运算"""
        return a - b
    
    def multiply(self, a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
        """乘法运算"""
        return a * b
    
    def divide(self, a: Union[int, float], b: Union[int, float]) -> float:
        """除法运算"""
        if b == 0:
            raise ValueError("除数不能为零")
        return a / b
    
    def power(self, base: Union[int, float], exponent: Union[int, float]) -> Union[int, float]:
        """幂运算"""
        return base ** exponent

def fibonacci(n: int) -> int:
    """计算斐波那契数列第 n 项"""
    if n < 0:
        raise ValueError("n 必须为非负整数")
    if n in (0, 1):
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
'''
        
        # pytest 测试示例
        pytest_tests = '''"""
使用 pytest 的测试示例
"""
import pytest
from calculator import Calculator, fibonacci

class TestCalculator:
    """计算器测试类"""
    
    def setup_method(self):
        """每个测试方法执行前的准备"""
        self.calc = Calculator()
    
    def test_add(self):
        """测试加法"""
        assert self.calc.add(2, 3) == 5
        assert self.calc.add(-1, 1) == 0
        assert self.calc.add(0.1, 0.2) == pytest.approx(0.3)
    
    def test_subtract(self):
        """测试减法"""
        assert self.calc.subtract(5, 3) == 2
        assert self.calc.subtract(0, 5) == -5
    
    def test_multiply(self):
        """测试乘法"""
        assert self.calc.multiply(3, 4) == 12
        assert self.calc.multiply(-2, 3) == -6
        assert self.calc.multiply(0, 100) == 0
    
    def test_divide(self):
        """测试除法"""
        assert self.calc.divide(6, 2) == 3
        assert self.calc.divide(1, 3) == pytest.approx(0.3333, abs=0.001)
    
    def test_divide_by_zero(self):
        """测试除零异常"""
        with pytest.raises(ValueError, match="除数不能为零"):
            self.calc.divide(5, 0)
    
    def test_power(self):
        """测试幂运算"""
        assert self.calc.power(2, 3) == 8
        assert self.calc.power(5, 0) == 1
        assert self.calc.power(4, 0.5) == 2

class TestFibonacci:
    """斐波那契数列测试类"""
    
    @pytest.mark.parametrize("n,expected", [
        (0, 0),
        (1, 1),
        (2, 1),
        (3, 2),
        (4, 3),
        (5, 5),
        (10, 55),
    ])
    def test_fibonacci_values(self, n, expected):
        """参数化测试斐波那契数列"""
        assert fibonacci(n) == expected
    
    def test_fibonacci_negative_input(self):
        """测试负数输入"""
        with pytest.raises(ValueError, match="n 必须为非负整数"):
            fibonacci(-1)
    
    @pytest.mark.slow
    def test_fibonacci_large_number(self):
        """测试大数计算（标记为慢速测试）"""
        result = fibonacci(20)
        assert result == 6765
'''
        
        # unittest 测试示例
        unittest_tests = '''"""
使用 unittest 的测试示例
"""
import unittest
from calculator import Calculator, fibonacci

class TestCalculator(unittest.TestCase):
    """计算器测试类"""
    
    def setUp(self):
        """每个测试方法执行前的准备"""
        self.calc = Calculator()
    
    def test_add(self):
        """测试加法"""
        self.assertEqual(self.calc.add(2, 3), 5)
        self.assertEqual(self.calc.add(-1, 1), 0)
        self.assertAlmostEqual(self.calc.add(0.1, 0.2), 0.3, places=7)
    
    def test_divide_by_zero(self):
        """测试除零异常"""
        with self.assertRaises(ValueError) as context:
            self.calc.divide(5, 0)
        self.assertIn("除数不能为零", str(context.exception))
    
    def tearDown(self):
        """每个测试方法执行后的清理"""
        pass

class TestFibonacci(unittest.TestCase):
    """斐波那契数列测试类"""
    
    def test_fibonacci_base_cases(self):
        """测试基础情况"""
        self.assertEqual(fibonacci(0), 0)
        self.assertEqual(fibonacci(1), 1)
    
    def test_fibonacci_sequence(self):
        """测试数列计算"""
        expected_values = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
        for i, expected in enumerate(expected_values):
            with self.subTest(i=i):
                self.assertEqual(fibonacci(i), expected)

if __name__ == '__main__':
    unittest.main()
'''
        
        # 写入文件
        (self.project_path / "calculator.py").write_text(main_code, encoding='utf-8')
        (self.project_path / "test_calculator_pytest.py").write_text(pytest_tests, encoding='utf-8')
        (self.project_path / "test_calculator_unittest.py").write_text(unittest_tests, encoding='utf-8')
        
        print("✅ 测试示例文件已创建")
        print("  - calculator.py (被测试的代码)")
        print("  - test_calculator_pytest.py (pytest 测试)")
        print("  - test_calculator_unittest.py (unittest 测试)")
    
    def create_pytest_config(self):
        """创建 pytest 配置文件"""
        
        pytest_ini = '''[tool.pytest.ini_options]
minversion = "6.0"
addopts = "-ra -q --strict-markers --disable-warnings"
testpaths = ["tests"]
markers = [
    "slow: marks tests as slow (deselect with '-m \"not slow\"')",
    "integration: marks tests as integration tests",
    "unit: marks tests as unit tests",
]
filterwarnings = [
    "ignore::UserWarning",
    "ignore::DeprecationWarning",
]
'''
        
        conftest_py = '''"""
pytest 配置文件
"""
import pytest

@pytest.fixture
def sample_data():
    """示例数据 fixture"""
    return {
        "numbers": [1, 2, 3, 4, 5],
        "strings": ["hello", "world", "pytest"],
        "config": {"debug": True, "timeout": 30}
    }

@pytest.fixture(scope="session")
def database_connection():
    """模拟数据库连接 fixture"""
    print("\\n设置数据库连接...")
    connection = "mock_db_connection"
    yield connection
    print("\\n关闭数据库连接...")

@pytest.fixture(autouse=True)
def setup_teardown():
    """自动执行的 fixture"""
    # setup
    print("\\n🔧 测试前准备...")
    yield
    # teardown
    print("🧹 测试后清理...")
'''
        
        # 写入配置文件
        (self.project_path / "pyproject.toml").write_text(
            (self.project_path / "pyproject.toml").read_text() + pytest_ini,
            encoding='utf-8'
        )
        (self.project_path / "conftest.py").write_text(conftest_py, encoding='utf-8')
        
        print("✅ pytest 配置文件已创建")
        print("  - pyproject.toml (添加了 pytest 配置)")
        print("  - conftest.py (pytest fixtures)")
    
    def demonstrate_debugging(self):
        """演示调试技巧"""
        print("\n=== 调试技巧演示 ===")
        
        # 创建包含调试点的示例代码
        debug_example = '''"""
调试示例代码
"""
import pdb

def complex_calculation(data):
    """复杂计算函数 - 包含调试点"""
    result = 0
    
    for i, item in enumerate(data):
        # 设置调试断点
        if i == 3:
            pdb.set_trace()  # 在第4次迭代时停止
        
        if isinstance(item, (int, float)):
            result += item * 2
        elif isinstance(item, str):
            result += len(item)
    
    return result

def fibonacci_with_debug(n):
    """带调试信息的斐波那契函数"""
    print(f"计算 fibonacci({n})")
    
    if n < 0:
        raise ValueError("n 必须为非负整数")
    if n in (0, 1):
        print(f"基础情况: fibonacci({n}) = {n}")
        return n
    
    # 递归计算
    result = fibonacci_with_debug(n - 1) + fibonacci_with_debug(n - 2)
    print(f"fibonacci({n}) = {result}")
    return result

if __name__ == "__main__":
    # 测试数据
    test_data = [1, 2, "hello", 3.5, "world"]
    
    print("普通执行:")
    result = complex_calculation(test_data.copy())
    print(f"结果: {result}")
    
    print("\\n斐波那契调试:")
    fib_result = fibonacci_with_debug(5)
    print(f"最终结果: {fib_result}")
'''
        
        (self.project_path / "debug_example.py").write_text(debug_example, encoding='utf-8')
        
        print("✅ 调试示例文件已创建: debug_example.py")
        print("\n🔍 调试技巧说明:")
        print("1. 使用 pdb.set_trace() 设置断点")
        print("2. 运行时使用 python -m pdb script.py 启动调试")
        print("3. 常用调试命令:")
        print("   - n (next): 执行下一行")
        print("   - s (step): 步入函数")
        print("   - c (continue): 继续执行")
        print("   - l (list): 显示当前代码")
        print("   - p variable_name: 打印变量值")
        print("   - q (quit): 退出调试器")

# 测试框架演示
testing_demo = TestingFrameworkDemo()

# 创建测试示例
testing_demo.create_test_examples()

# 创建 pytest 配置
testing_demo.create_pytest_config()

# 演示调试技巧
testing_demo.demonstrate_debugging()

print(f"\n📊 测试执行示例命令:")
print(f"  pytest test_calculator_pytest.py -v")
print(f"  python -m unittest test_calculator_unittest.py")
print(f"  pytest --cov=calculator test_calculator_pytest.py")
print(f"  pytest -m 'not slow' test_calculator_pytest.py")


## 4. 版本控制和 CI/CD 基础

### Git 工作流最佳实践

1. **分支策略**
   - `main/master`: 生产环境代码
   - `develop`: 开发分支
   - `feature/*`: 功能分支
   - `hotfix/*`: 紧急修复分支

2. **提交规范**
   ```
   type(scope): subject
   
   body
   
   footer
   ```
   
   常见 type：
   - `feat`: 新功能
   - `fix`: 错误修复
   - `docs`: 文档更新
   - `style`: 代码格式（不影响功能）
   - `refactor`: 代码重构
   - `test`: 测试相关
   - `chore`: 构建工具、辅助工具

### CI/CD 平台对比

| 平台 | 优势 | 适用场景 |
|------|------|----------|
| **GitHub Actions** | 与 GitHub 深度集成 | 开源项目、小团队 |
| **GitLab CI/CD** | 功能完整，自托管 | 企业级应用 |
| **Jenkins** | 高度可定制 | 复杂部署需求 |
| **CircleCI** | 配置简单 | 快速上手 |

### Docker 容器化

容器化是现代应用部署的标准方式：
- **一致性**：开发、测试、生产环境一致
- **可移植性**：跨平台部署
- **可扩展性**：容易水平扩展
- **资源隔离**：进程间隔离


In [ ]:
class DevOpsConfigGenerator:
    """
    DevOps 配置生成器 - 创建 CI/CD 和部署配置文件
    """
    
    def __init__(self):
        self.project_path = Path.cwd()
    
    def create_github_actions_workflow(self):
        """创建 GitHub Actions 工作流配置"""
        
        workflow_content = '''name: Python Backend CI/CD

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: [3.8, 3.9, 3.10, 3.11]

    steps:
    - uses: actions/checkout@v3
    
    - name: Set up Python ${{ matrix.python-version }}
      uses: actions/setup-python@v4
      with:
        python-version: ${{ matrix.python-version }}
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install flake8 pytest pytest-cov black mypy bandit
        if [ -f requirements.txt ]; then pip install -r requirements.txt; fi
    
    - name: Lint with flake8
      run: |
        # stop the build if there are Python syntax errors or undefined names
        flake8 . --count --select=E9,F63,F7,F82 --show-source --statistics
        # exit-zero treats all errors as warnings. The GitHub editor is 127 chars wide
        flake8 . --count --exit-zero --max-complexity=10 --max-line-length=127 --statistics
    
    - name: Check code formatting with Black
      run: black --check .
    
    - name: Type check with mypy
      run: mypy .
      continue-on-error: true
    
    - name: Security check with bandit
      run: bandit -r . -f json -o bandit-report.json
      continue-on-error: true
    
    - name: Test with pytest
      run: |
        pytest --cov=./ --cov-report=xml
    
    - name: Upload coverage to Codecov
      uses: codecov/codecov-action@v3
      with:
        file: ./coverage.xml
        flags: unittests
        name: codecov-umbrella

  build-and-deploy:
    needs: test
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    
    steps:
    - uses: actions/checkout@v3
    
    - name: Build Docker image
      run: |
        docker build -t myapp:${{ github.sha }} .
        docker tag myapp:${{ github.sha }} myapp:latest
    
    - name: Deploy to staging
      run: |
        echo "Deploying to staging environment..."
        # Add your deployment commands here
'''
        
        # 创建 .github/workflows 目录
        workflow_dir = self.project_path / ".github" / "workflows"
        workflow_dir.mkdir(parents=True, exist_ok=True)
        
        # 写入工作流文件
        (workflow_dir / "ci-cd.yml").write_text(workflow_content, encoding='utf-8')
        
        print("✅ GitHub Actions 工作流已创建")
        print("  - .github/workflows/ci-cd.yml")
    
    def create_dockerfile(self):
        """创建 Dockerfile"""
        
        dockerfile_content = '''# 使用官方 Python 运行时作为基础镜像
FROM python:3.11-slim

# 设置工作目录
WORKDIR /app

# 设置环境变量
ENV PYTHONDONTWRITEBYTECODE=1 \\
    PYTHONUNBUFFERED=1 \\
    PYTHONPATH=/app

# 安装系统依赖
RUN apt-get update \\
    && apt-get install -y --no-install-recommends \\
        build-essential \\
        curl \\
    && rm -rf /var/lib/apt/lists/*

# 复制 requirements 文件
COPY requirements.txt .

# 安装 Python 依赖
RUN pip install --no-cache-dir --upgrade pip \\
    && pip install --no-cache-dir -r requirements.txt

# 复制项目文件
COPY . .

# 创建非 root 用户
RUN adduser --disabled-password --gecos '' appuser \\
    && chown -R appuser:appuser /app
USER appuser

# 暴露端口
EXPOSE 8000

# 健康检查
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \\
    CMD curl -f http://localhost:8000/health || exit 1

# 启动命令
CMD ["python", "-m", "uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
'''
        
        # 创建 .dockerignore
        dockerignore_content = '''__pycache__
*.pyc
*.pyo
*.pyd
.Python
env/
.venv/
venv/
.env
.git
.gitignore
README.md
Dockerfile
.dockerignore
tests/
.coverage
.pytest_cache/
.mypy_cache/
'''
        
        (self.project_path / "Dockerfile").write_text(dockerfile_content, encoding='utf-8')
        (self.project_path / ".dockerignore").write_text(dockerignore_content, encoding='utf-8')
        
        print("✅ Docker 配置文件已创建")
        print("  - Dockerfile")
        print("  - .dockerignore")
    
    def create_docker_compose(self):
        """创建 Docker Compose 配置"""
        
        compose_content = '''version: '3.8'

services:
  web:
    build: .
    ports:
      - "8000:8000"
    environment:
      - DEBUG=False
      - DATABASE_URL=postgresql://user:password@db:5432/myapp
      - REDIS_URL=redis://redis:6379/0
    depends_on:
      - db
      - redis
    volumes:
      - ./logs:/app/logs
    restart: unless-stopped

  db:
    image: postgres:15-alpine
    environment:
      - POSTGRES_DB=myapp
      - POSTGRES_USER=user
      - POSTGRES_PASSWORD=password
    volumes:
      - postgres_data:/var/lib/postgresql/data
      - ./init.sql:/docker-entrypoint-initdb.d/init.sql
    ports:
      - "5432:5432"
    restart: unless-stopped

  redis:
    image: redis:7-alpine
    ports:
      - "6379:6379"
    volumes:
      - redis_data:/data
    restart: unless-stopped

  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
      - "443:443"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf
      - ./ssl:/etc/nginx/ssl
    depends_on:
      - web
    restart: unless-stopped

volumes:
  postgres_data:
  redis_data:
'''
        
        (self.project_path / "docker-compose.yml").write_text(compose_content, encoding='utf-8')
        
        # 创建开发环境的 Docker Compose
        dev_compose_content = '''version: '3.8'

services:
  web:
    build: .
    ports:
      - "8000:8000"
    environment:
      - DEBUG=True
      - DATABASE_URL=postgresql://user:password@db:5432/myapp_dev
    depends_on:
      - db
    volumes:
      - .:/app
      - ./logs:/app/logs
    command: python -m uvicorn main:app --host 0.0.0.0 --port 8000 --reload

  db:
    image: postgres:15-alpine
    environment:
      - POSTGRES_DB=myapp_dev
      - POSTGRES_USER=user
      - POSTGRES_PASSWORD=password
    volumes:
      - postgres_dev_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"

volumes:
  postgres_dev_data:
'''
        
        (self.project_path / "docker-compose.dev.yml").write_text(dev_compose_content, encoding='utf-8')
        
        print("✅ Docker Compose 配置已创建")
        print("  - docker-compose.yml (生产环境)")
        print("  - docker-compose.dev.yml (开发环境)")
    
    def create_makefile(self):
        """创建 Makefile 简化常用命令"""
        
        makefile_content = '''# Makefile for Python Backend Project

.PHONY: help install test lint format type-check security-check clean docker-build docker-run

# 默认目标
help:
	@echo "Available commands:"
	@echo "  install      - Install dependencies"
	@echo "  test         - Run tests"
	@echo "  lint         - Run linter"
	@echo "  format       - Format code"
	@echo "  type-check   - Run type checker"
	@echo "  security-check - Run security check"
	@echo "  clean        - Clean temporary files"
	@echo "  docker-build - Build Docker image"
	@echo "  docker-run   - Run with Docker Compose"

# 安装依赖
install:
	pip install --upgrade pip
	pip install -r requirements.txt
	pip install -r requirements-dev.txt

# 运行测试
test:
	pytest --cov=./ --cov-report=html --cov-report=term-missing

# 代码检查
lint:
	flake8 .
	pylint **/*.py

# 代码格式化
format:
	black .
	isort .

# 类型检查
type-check:
	mypy .

# 安全检查
security-check:
	bandit -r . -f json -o bandit-report.json

# 清理文件
clean:
	find . -type f -name "*.pyc" -delete
	find . -type d -name "__pycache__" -delete
	find . -type d -name "*.egg-info" -exec rm -rf {} +
	rm -rf .coverage htmlcov/ .pytest_cache/ .mypy_cache/

# Docker 相关
docker-build:
	docker build -t myapp:latest .

docker-run:
	docker-compose up -d

docker-dev:
	docker-compose -f docker-compose.dev.yml up

docker-stop:
	docker-compose down

# 完整的 CI 流程
ci: format lint type-check security-check test
	@echo "✅ All CI checks passed!"

# 部署准备
deploy-prep: ci docker-build
	@echo "🚀 Ready for deployment!"
'''
        
        (self.project_path / "Makefile").write_text(makefile_content, encoding='utf-8')
        
        print("✅ Makefile 已创建")
        print("  使用 'make help' 查看所有可用命令")

# 生成 DevOps 配置
devops_generator = DevOpsConfigGenerator()

print("\n=== DevOps 配置生成 ===")

# 创建 GitHub Actions 工作流
devops_generator.create_github_actions_workflow()

# 创建 Docker 配置
devops_generator.create_dockerfile()
devops_generator.create_docker_compose()

# 创建 Makefile
devops_generator.create_makefile()

print(f"\n🚀 DevOps 配置文件创建完成！")
print(f"")
print(f"📋 下一步操作建议:")
print(f"1. 提交代码到 Git 仓库")
print(f"2. 推送到 GitHub 触发 CI/CD")
print(f"3. 使用 'make docker-dev' 启动开发环境")
print(f"4. 使用 'make ci' 本地运行完整检查")

# 清理演示文件
print(f"\n🧹 清理演示文件...")
demo_files = [
    "bad_example.py", "good_example.py", "debug_example.py",
    "calculator.py", "test_calculator_pytest.py", 
    "test_calculator_unittest.py", "conftest.py"
]

for file in demo_files:
    file_path = Path.cwd() / file
    if file_path.exists():
        file_path.unlink()
        print(f"  已删除: {file}")

print(f"\n✅ 工具链教程完成！您现在拥有了完整的 Python 后端开发环境。")


## 5. 工具链最佳实践总结

### 🏗️ 开发环境搭建
- ✅ 使用虚拟环境隔离项目依赖
- ✅ 选择合适的包管理工具（pip/pipenv/poetry）
- ✅ 配置统一的开发环境
- ✅ 版本控制所有配置文件

### 🔍 代码质量保证
- ✅ 代码格式化：Black（无需配置，团队统一）
- ✅ 代码检查：Flake8 + Pylint（风格和质量）
- ✅ 类型检查：mypy（静态类型分析）
- ✅ 安全扫描：Bandit（漏洞检测）
- ✅ 自动化：pre-commit hooks

### 🧪 测试策略
- ✅ 单元测试：pytest（现代化、插件丰富）
- ✅ 覆盖率：pytest-cov（确保测试完整性）
- ✅ 集成测试：API 和数据库测试
- ✅ 性能测试：locust 或 pytest-benchmark

### 🐛 调试和性能
- ✅ 调试工具：pdb/ipdb 命令行调试
- ✅ 性能分析：cProfile、line_profiler
- ✅ 内存分析：memory_profiler
- ✅ 日志记录：结构化日志（structlog）

### 🚀 部署和运维
- ✅ 容器化：Docker + Docker Compose
- ✅ CI/CD：GitHub Actions 自动化流程
- ✅ 监控：日志聚合和性能监控
- ✅ 环境管理：开发/测试/生产环境分离

### 📊 推荐工具组合

#### 新手入门套装
```bash
pip + venv + black + flake8 + pytest
```

#### 中级开发套装
```bash
pipenv + black + flake8 + mypy + pytest + pre-commit
```

#### 高级生产套装
```bash
poetry + black + ruff + mypy + pytest + Docker + GitHub Actions
```

### 🔄 日常工作流

1. **开始新项目**
   ```bash
   mkdir my-project && cd my-project
   python -m venv .venv
   source .venv/bin/activate  # Windows: .venv\Scripts\activate
   pip install black flake8 pytest
   ```

2. **编写代码**
   ```bash
   # 编写代码
   # 运行格式化
   black .
   # 检查代码质量
   flake8 .
   # 运行测试
   pytest
   ```

3. **提交代码**
   ```bash
   git add .
   git commit -m "feat: add new feature"
   git push origin feature-branch
   ```

4. **生产部署**
   ```bash
   make ci  # 本地验证
   docker build -t myapp:latest .
   docker-compose up -d
   ```

### 🎯 进阶学习路径

1. **掌握现代工具**
   - 学习 `ruff`（更快的 linter）
   - 探索 `uv`（更快的包管理器）
   - 使用 `hatch` 或 `pdm`（新一代构建工具）

2. **深入 DevOps**
   - Kubernetes 容器编排
   - 监控系统（Prometheus + Grafana）
   - 日志系统（ELK Stack）

3. **性能优化**
   - 异步编程（asyncio）
   - 缓存策略（Redis）
   - 数据库优化

4. **团队协作**
   - 代码审查流程
   - 文档生成（Sphinx）
   - API 文档（OpenAPI/Swagger）

## 总结

工具链是提高开发效率和代码质量的关键。一个好的工具链应该：

- **自动化重复任务**：减少手工操作
- **保证代码质量**：统一标准和规范
- **提高团队协作**：统一的开发环境
- **支持快速迭代**：自动化测试和部署

> 💡 **记住**：工具是为了服务开发，而不是为了工具而工具。选择适合团队规模和项目需求的工具组合，持续改进和优化工作流程。

### 下一步建议

1. **立即行动**：为当前项目配置基础工具链
2. **逐步完善**：根据项目需要添加更多工具
3. **团队推广**：与团队分享和统一工具链
4. **持续学习**：关注新工具和最佳实践

🎉 **恭喜！您已经掌握了现代 Python 后端开发的完整工具链！**
